In [1]:
import nibabel as nib
import numpy as np
import cv2
from skimage.measure import label, regionprops
import os

def is_path_string(input_obj):
    """
    判断输入是否是一个路径字符串。
    """
    return isinstance(input_obj, str) and os.path.isfile(input_obj)

def retain_threshold_range(volume, lower_val, upper_val, kernel_size=(15, 15)):
    """
    保留3D图像中在指定阈值范围内的像素，并进行形态学开运算去除噪声。

    参数:
    volume -- 输入3D图像 (numpy数组)
    lower_val -- 阈值范围的下限
    upper_val -- 阈值范围的上限
    kernel_size -- 形态学操作的核大小，默认为(5, 5)

    返回:
    mask -- 处理后的掩码3D图像
    """
    # 应用阈值范围
    mask = np.zeros_like(volume, dtype=np.uint8)
    mask[(volume >= lower_val) & (volume <= upper_val)] = 255
    
    # 创建形态学操作的核
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    
    # 进行形态学开运算去除噪声
    for i in range(mask.shape[2]):
        mask[:, :, i] = cv2.morphologyEx(mask[:, :, i], cv2.MORPH_OPEN, kernel)
    
    return mask

def get_largest_connected_component(volume):
    """
    获取3D图像中掩码的最大连通区域，并填充内部孔洞。

    参数:
    volume -- 输入掩码3D图像 (numpy数组)

    返回:
    largest_component_mask -- 最大连通区域的掩码3D图像
    """
    # 找到所有连通组件
    labeled_volume, num_labels = label(volume, connectivity=3, return_num=True)

    # 找到体积最大的连通组件，排除背景 (标签0)
    largest_label = 1
    max_area = 0
    for region in regionprops(labeled_volume):
        if region.area > max_area:
            max_area = region.area
            largest_label = region.label

    # 创建一个掩码，仅保留最大连通组件
    largest_component_mask = np.zeros_like(volume, dtype=np.uint8)
    largest_component_mask[labeled_volume == largest_label] = 255

    # 填充最大连通区域内部的孔洞
    # 默认（5，5）
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (50, 50))
    for i in range(largest_component_mask.shape[2]):
        largest_component_mask[:, :, i] = cv2.morphologyEx(largest_component_mask[:, :, i], cv2.MORPH_CLOSE, kernel)

    return largest_component_mask

def apply_mask(volume, mask):
    """
    将掩码应用到原始图像，非遮盖区域保持原始图像的灰度值，遮盖区域设为0。

    参数:
    volume -- 原始3D图像 (numpy数组)
    mask -- 掩码3D图像 (numpy数组)

    返回:
    masked_volume -- 应用掩码后的3D图像
    """
    masked_volume = np.where(mask == 255, volume, 0)
    return masked_volume

def modify_nifti_origin(fixed_image_path, moving_image_path, output_file=None):
    """
    修改NIfTI图像的原点位置。

    参数：
    moving_image_path (str): 移动NIfTI文件的路径。
    fixed_image_path (str): 固定NIfTI文件的路径。
    output_file (str): 输出修改后NIfTI文件的路径。
    """
    # 加载移动NIfTI图像以获取其Affine矩阵
    moving_img = nib.load(moving_image_path)
    moving_affine = moving_img.affine
    
    # 加载固定NIfTI图像
    fixed_img = nib.load(fixed_image_path)

    # 获取图像数据和头文件
    fixed_data = fixed_img.get_fdata()
    fixed_header = fixed_img.header

    # 使用移动图像的Affine矩阵创建新的NIfTI图像
    new_img = nib.Nifti1Image(fixed_data, moving_affine, fixed_header)

    # 如果提供了输出文件路径，则保存新的NIfTI图像
    if output_file:
        nib.save(new_img, output_file)
        print(f"Modified NIfTI image saved as {output_file}")
    
    return new_img

# 去除边上的黑边    
def remove_black_edges(nib_obj, output_file):
    # 读取NIfTI图像
    img = nib_obj
    data = nib_obj.get_fdata()

    # 找到非黑色（非零）区域的边界
    non_zero_coords = np.argwhere(data)
    min_coords = non_zero_coords.min(axis=0)
    max_coords = non_zero_coords.max(axis=0)

    # 提取非黑色区域
    cropped_data = data[min_coords[0]:max_coords[0]+1,
                        min_coords[1]:max_coords[1]+1,
                        min_coords[2]:max_coords[2]+1]

    # 创建新的NIfTI图像
    return nib.Nifti1Image(cropped_data, img.affine)
# 修改NIfTI图像的Affine矩阵，使其与目标Affine矩阵一致。
def modify_nifti_affine(input_file, target_affine, output_file = None):
    """
    修改NIfTI图像的Affine矩阵，使其与目标Affine矩阵一致。

    参数：
    input_file (str): 输入NIfTI文件的路径。
    output_file (str): 输出修改后NIfTI文件的路径。
    target_affine (numpy.ndarray): 目标Affine矩阵。
    """
    # 加载NIfTI图像
    img = nib.load(input_file)

    # 获取图像数据和头文件
    data = img.get_fdata()
    header = img.header

    # 创建新的NIfTI图像，使用目标Affine矩阵
    new_img = nib.Nifti1Image(data, target_affine, header)

    # 保存新的NIfTI图像
    # nib.save(new_img, output_file)
    print(f"New affine matrix: \n{target_affine}")
    
    # 打印新的Affine矩阵
    print(f"Modified NIfTI image saved as {output_file}")
    return new_img

    
    
def modify_nifti_affine_ants(ants_obj, target_affine_path,output_image_path = None ):
    """
    修改NIfTI图像的Affine矩阵，使其与目标Affine矩阵一致。
    将输入图像的仿射矩阵修改为目标图像的仿射矩阵
    参数：
    input_file (str): 输入NIfTI文件的路径。或
    target_affine (numpy.ndarray): 目标Affine矩阵。
    
    """
    input_image = {}
    if is_path_string(ants_obj):
        input_image = nib.load(ants_obj)
        # 处理路径字符串
    else:
        input_image = ants_obj    
     # 读取目标仿射矩阵
    target_affine_image = nib.load(target_affine_path)
    target_affine = target_affine_image.affine
    
    # 获取输入图像的仿射矩阵
    input_affine = input_image.get_affine()
    
    # 修改输入图像的仿射矩阵为目标仿射矩阵
    input_image.set_affine(target_affine)
    
    # 保存修改后的图像
    if not output_image_path:
        output_image_path
    print(f"Modified affine matrix of  saved to {output_image_path}")


    return input_image

def removeBone(nii_path,resultPath=None,remove_black_edges=False):
    nii_img = {}
    try:    
        if is_path_string(nii_path):
            nii_img = nib.load(nii_path)
            # 处理路径字符串
        else:
            nii_img = nii_path      
        
        volume = nii_img.get_fdata()
        # 设定阈值范围
        lower_val = -10
        upper_val = 70
        # 阈值处理
        mask = retain_threshold_range(volume, lower_val, upper_val)
        # 获取最大连通区域
        largest_component_mask = get_largest_connected_component(mask)
        # 应用掩码
        masked_volume = apply_mask(volume, largest_component_mask)
        # 保存结果
        masked_img = nib.Nifti1Image(masked_volume, nii_img.affine)
        nib.save(masked_img, "temp.nii.gz")
        # if not remove_black_edges:
        #     remove_black_edges(masked_img)
            
        # if  not resultPath:
        #     nib.save(masked_img, resultPath)
        print(f" removebone finished!\n")
        return masked_img
        
    except FileNotFoundError:
        print(f"文件未找到: {nii_path}")
    except nib.filebasedimages.ImageFileError:
        print(f"无法加载 NIfTI 文件: {nii_path}")
    except Exception as e:
        print(f"处理过程中发生错误: {e}")

In [2]:
# removeBone(r"C:\Users\ruxua\Desktop\normal\L40.nii.gz")

In [3]:
import nibabel as nib
import numpy as np
import ants

def convert_nibabel_obj_to_ants(nib_image):
    """
    将 NiBabel 的图像对象转换为 ANTsPy 的图像对象。

    参数:
    nib_image (nibabel.Nifti1Image): NiBabel 图像对象。

    返回:
    ants.ANTsImage: 转换后的 ANTsPy 图像对象。
    """
    # 提取图像数据和仿射矩阵
    nib_data = nib_image.get_fdata()
    nib_affine = nib_image.affine
    
    # 创建ANTsPy图像对象
    ants_image = ants.from_numpy(nib_data, origin=nib_affine[:3, 3], spacing=nib_image.header.get_zooms())
    
    return ants_image

def load_nifti(file_path):
    """加载NIfTI文件并返回数据和仿射矩阵"""
    img = nib.load(file_path)
    data = img.get_fdata()
    affine = img.affine
    return data, affine

def save_nifti(data, affine, file_path):
    """保存NIfTI文件"""
    img = nib.Nifti1Image(data, affine)
    nib.save(img, file_path)

def register_images(fixed_image_path, moving_image_path):
    
    
    # """使用ANTs进行图像配准"""
    # fixed_image = ants.image_read(fixed_image_path)
    # moving_image = ants.image_read(moving_image_path)
    # # 检查是否成功读取图像
    # if fixed_image is None or moving_image is None:
    #     print("Failed to read one or both images. Exiting...")
    #     exit(1)
        
    #     # 打印图像信息
    # print(f"CT Image: {fixed_image}")
    # print(f"MRI Image: {moving_image}")

    # 执行配准
    try:
        # 配准前准备
        # 改变位置
        # 去骨
        print("开始去骨")
        removeBone(moving_image_path)
        print("去骨ok")
        # nib 对象转ants对象
        moving_image = ants.image_read("temp.nii.gz")
        fixed_image = ants.image_read(fixed_image_path)
        # 检查是否成功读取图像
        if fixed_image is None or moving_image is None:
            print("Failed to read one or both images. Exiting...")
            exit(1)            
            # 打印图像信息
        print(f"fixed_image: {fixed_image}")
        print(f"moving_image: {moving_image}")
        print("开始配准")  
        transform = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform='SyN') 
        print(f"Type of registration_result: {type(transform)}")
        print(f"Length of registration_result: {len(transform)}")
        if transform:
            print("The returned dictionary is not empty. Contents:")
            for key, value in transform.items():
                print(f"{key}: {value}")
        else:
            print("The returned dictionary is empty.")        
        print("配准ok")   
        # 保存变换参数
        # 检查 transform 中的变换是否存在
        return transform 
    except Exception as e:
        print(f"Error during registration: {e}")
        exit(1)



# 基准CT图像
fixed_image_path = r"C:\SynologyDrive\code\brain-seg\images\CTbase\20240804_1\L40nobone.nii.gz"
# 基准CT分区
functional_area_path = r"C:\SynologyDrive\code\brain-seg\images\CTbase\20240804_1\L40nobone7mask.nii.gz"

globalTF={}



In [4]:
# # 需要配准判断的图像
# moving_image_path = r"C:\Users\ruxuanyan\Desktop\pei\restored_ct_0\CQ500-CT-4.nii.gz"
# # 需要配准判断的图像的病变标签
# lesion_path = r"C:\Users\ruxuanyan\Desktop\pei\nn_registration_1\4.nii.gz"



In [5]:

# 配准图像
# transform = register_images(fixed_image_path, moving_image_path)
# transform['warpedmovout'].to_file("temp.nii.gz")

In [6]:
# transform['warpedmovout'].to_file("temp.nii.gz")

In [7]:
import ants
import numpy as np
import pandas as pd

# 病变标签映射表
lesion_label_map = {
    0: "background",
    1: "chuxue",
    2: "naoshi",
    3: "shuizhong",
    4: "zhuxue",
    5: "xia",
    6: "wai"
}

# 功能区标签映射表（你可以根据需要扩展或调整）
functional_area_label_map = {
    0: "Clear Label",
    200: "Deep_R",
    201: "Deep_L",
    202: "BrainStem_R",
    203: "Lobar_R",
    204: "Lobar_L",
    205: "BrainStem_L",
    206: "Cerebellum_R",
    207: "Cerebellum_L",
    208: "3rd4th ventricles_L",
    209: "Lateral ventricle_R",
    210: "Lateral ventricle_L",
    211: "3rd4th ventricles_R"
}
def register_images(fixed_image_path, moving_image_path):
    
    
    # """使用ANTs进行图像配准"""
    # fixed_image = ants.image_read(fixed_image_path)
    # moving_image = ants.image_read(moving_image_path)
    # # 检查是否成功读取图像
    # if fixed_image is None or moving_image is None:
    #     print("Failed to read one or both images. Exiting...")
    #     exit(1)
        
    #     # 打印图像信息
    # print(f"CT Image: {fixed_image}")
    # print(f"MRI Image: {moving_image}")

    # 执行配准
    try:
        # 配准前准备
        # 改变位置
        # 去骨
        print("开始去骨")
        removeBone(moving_image_path)
        print("去骨ok")
        # nib 对象转ants对象
        moving_image = ants.image_read("temp.nii.gz")
        fixed_image = ants.image_read(fixed_image_path)
        # 检查是否成功读取图像
        if fixed_image is None or moving_image is None:
            print("Failed to read one or both images. Exiting...")
            exit(1)            
            # 打印图像信息
        print(f"fixed_image: {fixed_image}")
        print(f"moving_image: {moving_image}")
        print("开始配准")  
        transform = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform='SyN') 
        print(f"Type of registration_result: {type(transform)}")
        print(f"Length of registration_result: {len(transform)}")
        if transform:
            print("The returned dictionary is not empty. Contents:")
            for key, value in transform.items():
                print(f"{key}: {value}")
        else:
            print("The returned dictionary is empty.")        
        print("配准ok")   
        # 保存变换参数
        # 检查 transform 中的变换是否存在
        return transform 
    except Exception as e:
        print(f"Error during registration: {e}")
        exit(1)


def apply_transform(moving_image, reference_image, transform):
    """应用变换到移动图像"""
    # transformed_image = ants.apply_transforms(fixed=reference_image, moving=moving_image, transformlist=transform['fwdtransforms'])
    transformed_image = ants.apply_transforms(fixed=reference_image, moving=moving_image, transformlist=transform['fwdtransforms'], interpolator='nearestNeighbor')
    return transformed_image

def map_lesion_to_functional_areas(lesion_path, functional_area_path, transform, output_path):
    lesion_image = ants.image_read(lesion_path)
    functional_area_image = ants.image_read(functional_area_path)
    
    # 转换为四舍五入后的整数类型
    original_lesion_data = np.round(lesion_image.numpy()).astype(int)
    original_unique_labels = np.unique(original_lesion_data)
    print(f"映射前的唯一标签数量: {len(original_unique_labels)}")
    print(f"映射前的唯一标签: {original_unique_labels}")

    transformed_lesion = apply_transform(lesion_image, functional_area_image, transform)
    lesion_data = np.round(transformed_lesion.numpy()).astype(int)
    
    functional_area_data = np.round(functional_area_image.numpy()).astype(int)

    # 打印映射后的唯一标签数量
    transformed_unique_labels = np.unique(lesion_data)
    print(f"映射后的唯一标签数量: {len(transformed_unique_labels)}")
    print(f"映射后的唯一标签: {transformed_unique_labels}")

    # 打印功能区的唯一标签
    functional_area_labels = np.unique(functional_area_data)
    print(f"功能区的唯一标签数量: {len(functional_area_labels)}")
    print(f"功能区的唯一标签: {functional_area_labels}")

    results = []

    for lesion_label in original_unique_labels:
        if lesion_label == 0:
            continue
        # 筛选出单独标签的总数量lesion_area
        lesion_area = np.sum(lesion_data == lesion_label)
        if lesion_area == 0:
            continue
        
        # 初始化所有功能区的比例为0
        ratios = {f"func_{func_label}": 0 for func_label in functional_area_labels if func_label != 0}
        
        # 在功能区算比例
        for func_label in functional_area_labels:
            # if func_label == 0:
            #     continue
            
            overlap = np.logical_and(lesion_data == lesion_label, functional_area_data == func_label)
            overlap_area = np.sum(overlap)
            
            ratio = overlap_area / lesion_area if lesion_area > 0 else 0
            ratios[f"func_{func_label}"] = ratio

        
        # 保存结果
        result = {
            "file_name": os.path.basename(lesion_path),
            "lesion": lesion_label,
            "total_area": lesion_area
        }
        result.update(ratios)
        results.append(result)
    
    if results:
        # 将结果保存为 DataFrame
        df = pd.DataFrame(results)
    else:
        # 如果结果为空，创建一个具有特定列的 DataFrame 并添加一行空值
        columns = ["file_name", "lesion", "total_area"] + [f"func_{label}" for label in functional_area_labels if label != 0]
        df = pd.DataFrame(columns=columns)
        empty_row = {col: 0 for col in columns}
        empty_row["file_name"] = os.path.basename(lesion_path)
        df = pd.concat([df, pd.DataFrame([empty_row])], ignore_index=True)
        print("没有找到任何有效的病变标签，生成一个具有特定列的空 DataFrame。")
    
    # 映射功能区标签
    df.rename(columns=functional_area_label_map, inplace=True)
    
    # 映射病变标签
    df['lesion'] = df['lesion'].map(lesion_label_map)
    
    # 保存为 Excel 文件
    output_excel_path = output_path.replace('.nii.gz', '.xlsx')
    df.to_excel(output_excel_path, index=False)
    print(f"结果已保存为 Excel 文件: {output_excel_path}")
    
    ants.image_write(transformed_lesion, output_path)
    print(f"映射后的病变图像已保存为: {output_path}")

    return df

def print_label_ratios(df):
    for _, row in df.iterrows():
        lesion_name = row['lesion']
        print(f'病变标签 {lesion_name} 在各功能区的比例:')
        for col in df.columns:
            if col not in ['lesion', 'total_overlap_area', 'total_area', 'file_name'] and not pd.isna(row[col]):
                print(f'  功能区 {col}: {row[col]:.4f}')

# # 示例调用
# output_path = 'path_to_save_transformed_lesion.nii.gz'

# df = map_lesion_to_functional_areas(lesion_path, functional_area_path, transform, output_path)
# print_label_ratios(df)



# 批量运行

In [8]:
import os
import ants
import pandas as pd
import numpy as np

def process_all_lesions(moving_image_folder, lesion_folder, functional_area_path, output_folder):
    """
    处理病变图像文件夹中的所有图像文件，并执行配准和映射操作。

    参数:
    moving_image_folder -- 移动图像文件夹路径
    lesion_folder -- 病变图像文件夹路径
    functional_area_path -- 功能区图像路径
    output_folder -- 输出文件夹路径
    """
    # 获取所有病变图像文件
    lesion_files = [f for f in os.listdir(lesion_folder) if f.endswith('.nii.gz')]
    
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)
    
    all_results = []

    for lesion_file in lesion_files:
        lesion_path = os.path.join(lesion_folder, lesion_file)
        
        # 构造移动图像文件的路径
        # moving_image_file = lesion_file.replace('.nii.gz', '.nii.gz')  # 这里假设移动图像的文件名与病变图像相同
        moving_image_file = "CQ500-CT-"+lesion_file
        moving_image_path = os.path.join(moving_image_folder, moving_image_file)

        if not os.path.isfile(moving_image_path):
            print(f"移动图像 {moving_image_path} 不存在，跳过.")
            continue

        # 构造输出文件的路径
        output_path = os.path.join(output_folder, lesion_file.replace('.nii.gz', '_transformed.nii.gz'))
        
        # 检查输出文件是否存在
        if os.path.isfile(output_path):
            print(f"输出文件 {output_path} 已存在，跳过.")
            continue

        # 配准图像
        try:
            transform = register_images(fixed_image_path, moving_image_path)
            output_path_ = output_path.replace('_transformed.nii.gz', '_image.nii.gz')
            transform['warpedmovout'].to_file(output_path_)
        except Exception as e:
            print(f"配准失败 {lesion_file}: {e}")
            continue

        # 映射病变标签到功能区
        try:
            df = map_lesion_to_functional_areas(lesion_path, functional_area_path, transform, output_path)
            all_results.append(df)
        except Exception as e:
            print(f"映射失败 {lesion_file}: {e}")
            continue

    # 合并所有结果
    if all_results:
        combined_df = pd.concat(all_results, ignore_index=True)
        excel_output_path = os.path.join(output_folder, 'combined_results.xlsx')
        combined_df.to_excel(excel_output_path, index=False)
        print(f"所有结果已保存为 Excel 文件: {excel_output_path}")

# 示例调用
# 基准CT图像
fixed_image_path = r"C:\SynologyDrive\code\brain-seg\images\CTbase\20240804_1\L40nobone.nii.gz"
# 基准CT分区
functional_area_path = r"C:\SynologyDrive\code\brain-seg\images\CTbase\20240804_1\L40nobone7mask.nii.gz"

moving_image_folder = r"C:\Users\ruxuanyan\Desktop\pei\restored_ct_1"
lesion_folder = r"C:\Users\ruxuanyan\Desktop\pei\nn_registration_1"
output_folder = r"C:\Users\ruxuanyan\Desktop\pei\processed_results"

process_all_lesions(moving_image_folder, lesion_folder, functional_area_path, output_folder)


开始去骨
 removebone finished!

去骨ok
fixed_image: ANTsImage (RPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 32)
	 Spacing    : (0.4297, 0.4297, 5.0)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [ 1.  0.  0.  0. -1.  0.  0.  0.  1.]

moving_image: ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (231, 231, 30)
	 Spacing    : (1.0, 1.0, 5.0)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [ 1.      0.      0.      0.      0.9877  0.1564  0.     -0.1564  0.9877]

开始配准
Type of registration_result: <class 'dict'>
Length of registration_result: 4
The returned dictionary is not empty. Contents:
warpedmovout: ANTsImage (RPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 32)
	 Spacing    : (0.4297, 0.4297, 5.0)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [ 1.  0.  0.  0. -1.  0.  0.  0.  1.]

warpedfixout: ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (231, 231, 30)
	 Spacing 

# 多线程加进度条

In [ ]:
import os
import ants
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm



def process_lesion(lesion_file, moving_image_folder, lesion_folder, functional_area_path, output_folder, fixed_image_path):
    try:
        lesion_path = os.path.join(lesion_folder, lesion_file)
        moving_image_file = "CQ500-CT-" + lesion_file
        moving_image_path = os.path.join(moving_image_folder, moving_image_file)

        if not os.path.isfile(moving_image_path):
            print(f"移动图像 {moving_image_path} 不存在，跳过.")
            return None

        # 配准图像
        transform = register_images(fixed_image_path, moving_image_path)

        # 映射病变标签到功能区
        output_path = os.path.join(output_folder, lesion_file.replace('.nii.gz', '_transformed.nii.gz'))
        df = map_lesion_to_functional_areas(lesion_path, functional_area_path, transform, output_path)
        print("完成一个")
        return df

    except Exception as e:
        print(f"处理失败 {lesion_file}: {e}")
        return None

def process_all_lesions(moving_image_folder, lesion_folder, functional_area_path, output_folder, fixed_image_path):
    lesion_files = [f for f in os.listdir(lesion_folder) if f.endswith('.nii.gz')]
    os.makedirs(output_folder, exist_ok=True)

    all_results = []

    with ThreadPoolExecutor(max_workers=8) as executor:  # 您可以根据需要调整最大线程数
        futures = {executor.submit(process_lesion, lesion_file, moving_image_folder, lesion_folder, functional_area_path, output_folder, fixed_image_path): lesion_file for lesion_file in lesion_files}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing lesions"):
            result = future.result()
            if result is not None:
                all_results.append(result)

    if all_results:
        combined_df = pd.concat(all_results, ignore_index=True)
        excel_output_path = os.path.join(output_folder, 'combined_results.xlsx')
        combined_df.to_excel(excel_output_path, index=False)
        print(f"所有结果已保存为 Excel 文件: {excel_output_path}")



# 示例调用
# 基准CT图像
fixed_image_path = r"C:\SynologyDrive\code\brain-seg\images\CTbase\20240804_0\L40nobone.nii.gz"
# 基准CT分区
functional_area_path = r"C:\SynologyDrive\code\brain-seg\images\CTbase\20240804_0\L40nobone7mask.nii.gz"

moving_image_folder = r"C:\Users\ruxuanyan\Desktop\pei\restored_ct_0"
lesion_folder = r"C:\Users\ruxuanyan\Desktop\pei\nn_registration_0"
output_folder = r"C:\Users\ruxuanyan\Desktop\pei\processed_results"

process_all_lesions(moving_image_folder, lesion_folder, functional_area_path, output_folder, fixed_image_path)
